In [40]:
import numpy as np

In [43]:
'''
The create_board() function simply returns a 14-length numpy array of all zeros. This will be the format that we will use to represent the state
of the board. Indeces 0-5 are player0's pieces and indeces 6-11 are player1's pieces, where 0-1 are small player0 pieces, 2-3 are medium player0
pieces, 4-5 are large player0 pieces, 6-7 small player1 pieces, 8-9 medium player1 pieces, 10-11 large player1 pieces. The eligible numbers in
these spots range from 0-9 where 0 means the piece has not been placed and 1-9 represent the spots on the board from top left to bottom right.
Index 12 is either 0 or 1 where 0 means it's player0's turn to play and 1 means it's player1's turn next (the 0 and 1 are why I call them player0
and 1 instead of 1 and 2), and index 13 represents the winning state of the board. The goal is to have index 13 range from -10 to 10 where 10
means the board is in a state where player0 has won and -10 means player1 has won, and then the scoring mechanism (still to be decided how this
will work) will determine the ranges from -9 to 9 and essentially represents who has the advantage of the current board state and how big is that
advantage (positive number is advantage for player0 and negative for player1).
'''
def create_board():
    return np.zeros(14, dtype = int)
board1 = create_board()
print(board1)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [44]:
# We have decided to tackle this research question in 2 stages. The overall Gobblet Gobblers game allows players to move pieces that they have
# already placed on the board to different spots, but we decided to simplify this for the first stage. So as of right now a player is not able
# to move a piece that they have already placed.
'''
The available_pieces() function takes in the current board and determines what available pieces the player whose turn it is can play. In stage 1
this is similar to tic-tac-toe where only non-placed pieces are available, but this should be updated in stage 2. In the updated version all
pieces should be available unless it's currently on the board and covered by a larger piece. This function returns a numpy array containing
the indeces of the available pieces for the current player.
'''
def available_pieces(board):
    if board[12] == 0:
        # np.where returns an array of the indeces that match the conditional statement. np.where actually returns a tuple that contains the 
        # array inside of it, so using [0] at the end of it will return the actual array.
        return np.where(board[0:6] == 0)[0]
    else:
        return np.where(board[6:12] == 0)[0] + 6
placeable_pieces = available_pieces(board1)
print(placeable_pieces)

[0 1 2 3 4 5]


In [45]:
# Enter results from available_pieces into this function
'''
The eliminate_redundant_pieces() function takes in the output array from available_pieces() and determines if the player has two pieces of the same
size. It then deletes all the second pieces of the same size from the available_pieces array. By only allowing the first piece to be placed of
matching-sized unplayed pieces, we can reduce the amount of possible boards in the beginning plays. If available_pieces = [0, 1, 4, 5] then
this function will return [0, 4].
'''
def eliminate_redundant_pieces(piece_set):
    # Checking for 2 conditions to be true in available_pieces array: 
    # piece is even number and the piece at (index + 1) is one number greater, if yes then delete the second occurence of the piece size
    pieces_with_redundancy = np.where((piece_set[:-1] % 2 == 0) & (piece_set[1:] == piece_set[:-1] + 1))[0]
    return np.delete(piece_set, pieces_with_redundancy + 1)

In [46]:
leftover_pieces = eliminate_redundant_pieces(available_pieces(board1))
print(leftover_pieces)

[0 2 4]


From here, we discovered 2 different approaches for how to populate a data structure to store all of the canonical boards:
1. Using a set for canonical boards allows the use of hashing for storage of the boards, so checking if boards exist within the set is very efficient and the time it takes to search for boards is not dependent on the size of the set. This approach requires checking if all 8 symmetries of each board we encounter exist within the canonical_boards set.
2. Using a list for canonical boards allows us to use a series of checks of symmetry for each board we encounter against each board inside of canonical_list. This approach requires using a for-loop to compare the new board to each canonical board, but the series of if, elif, else statements allow for quick checks of the 8 possible symmetries.  
Because the canonical boards data structure will become so large due to all the possible board states, running tests to find the more efficient process will be crucial. Having this data structure will greatly reduce the computing needed when the game theory tree is created since once we create the tree for one board we can automatically "play out" all boards that are symmetric to that board and there will be no need to recreate that tree.

In [47]:
'''
The switch_player() function simply switches the 13th element of the board from 0 to 1 or 1 to 0 to indicate it is the other player's turn.
'''
def switch_player(board):
    if board[12] == 0:
        board[12] = 1
    else:
        board[12] = 0

In [48]:
'''
These 7 arrays represent the transformation operations that are performed on a board to obtain the symmetric versions of that board. Because our
arrays are not set up in a format where we can use matrix multiplication to transform the boards, we have to coordinate each spot on the board to
a new spot depending on the type of transformation. The NumPy concept we use here is called 'array indexing' and essentially it takes in a board and
assigns each piece (indeces 0-11) with a new spot on the board that represents some type of transformation. It is set up like this:
rotation_90_mapping[board]. Essentially, it maps the current spot of a piece in the board array to the spot in the mapping array that has the index
of the current spot of the piece. Yeah I know that's confusing wording, it took me a while to get it too. If I have a 
                     current_board = [5, 0, 0, 0, 3, 0, 9, 0, 0, 0, 8, 0, 0, 0] then the resulting board from 
rotation_90_mapping[current_board] = [5, 0, 0, 0, 9, 0, 7, 0, 0, 0, 4, 0, 0, 0]. Since the 3rd index of rotation_90_mapping is 9 (look at
rotation_90_mapping below), then the piece in the 3rd spot from board array goes to spot 9, the piece in the 9th spot goes to spot 7, the piece in
the 8th spot goes to spot 4, etc. I would recommend pulling out a sheet of paper and and making a tic-tac-toe board state and trying these out to
visualize this concept better.
'''
rotation_90_mapping = np.array([0, 3, 6, 9, 2, 5, 8, 1, 4, 7])
rotation_180_mapping = np.array([0, 9, 8, 7, 6, 5, 4, 3, 2, 1])
rotation_270_mapping = np.array([0, 7, 4, 1, 8, 5, 2, 9, 6, 3])
vertical_reflection_mapping = np.array([0, 7, 8, 9, 4, 5, 6, 1, 2, 3])
horizontal_reflection_mapping = np.array([0, 3, 2, 1, 6, 5, 4, 9, 8, 7])
r90_vreflect_mapping = np.array([0, 9, 6, 3, 8, 5, 2, 7, 4, 1])
r90_hreflect_mapping = np.array([0, 1, 4, 7, 2, 5, 8, 3, 6, 9])

In [49]:
# This will be the data structure that we will use for the 'set' approach that we described earlier as the first approach to checking symmetry.
canonical_boards = set()

In [50]:
def reorder_pieces(board_array):
    for i in range(0, len(board_array), 2):
        if board_array[i+1] != 0 & (board_array[i] > board_array[i+1]):
            board_array[i:i+2].sort()
    return board_array

In [51]:
'''
The check_symmetry() function checks if our current board or any symmetric versions of our current board exist inside of the canonical_boards set,
then adds our current board to canonical_boards if none of them exist inside of it. Because canonical_boards is a set, it doesn't allow mutable
objects to be stored inside of it so we have to convert our board arrays into tuples (non-mutable) to check if they exist in canonical_boards
and to add them. We store the tuple versions of our 8 symmetric boards in a set of their own and use a set operation to efficiently check if there
is any overlap between that set and canonical_boards.
'''
def check_symmetry(board):
    symmetric_boards = {tuple(board),
                        # We only want to apply the array indexing to the players pieces in the board array (indeces 0-11) but we want to check for the
                        # entire board in canonical_boards, so we have to combine the transformed version of the pieces with the last 2 indeces
                        # using np.append
                        tuple(np.append(reorder_pieces(rotation_90_mapping[board[:12]]), board[12:])),
                        tuple(np.append(reorder_pieces(rotation_180_mapping[board[:12]]), board[12:])),
                        tuple(np.append(reorder_pieces(rotation_270_mapping[board[:12]]), board[12:])),
                        tuple(np.append(reorder_pieces(vertical_reflection_mapping[board[:12]]), board[12:])),
                        tuple(np.append(reorder_pieces(horizontal_reflection_mapping[board[:12]]), board[12:])),
                        tuple(np.append(reorder_pieces(r90_vreflect_mapping[board[:12]]), board[12:])),
                        tuple(np.append(reorder_pieces(r90_hreflect_mapping[board[:12]]), board[12:]))}
    
    # .isdisjoint() returns True if there are no common elements between symmetric_boards and canonical_boards and False if there are common elements
    if symmetric_boards.isdisjoint(canonical_boards):
        canonical_boards.add(tuple(board))

In [52]:
# This will be the data structure that we will use for the 'list' approach that was described earlier as the second approach to checking symmetry.
canonical_list = []

In [53]:
'''
The isNew() function accomplishes the same goal as check_symmetry() but achieves it differently. This isNew() function takes in a board state and
uses a for-loop to compare it individually to each board inside of canonical_list. To compare the board state to each canonical board, we use a series
of symmetry checks so that we can efficiently do two different things: 1. move to the next canonical board in the for-loop if the new board state is not
equal to or symmetric to the canonical board it is being compared with, and 2. the entire function returns False as soon as we find that the new board
state is equal to or symmetric to a canonical board. If we loop through every canonical board and no equalities/symmetries are found with the new board
state then the new board state is canonical and will be added to the canonical_list (not done inside of this function). isNew() returns False if the
board state is equal to or symmetric to a canonical board and True if it's a new board that we have not run into yet.
'''

# Once again I recommend pulling out a sheet of paper and working through these symmetry checks for a better understanding of what is going on
'''
Here's a quick run-through of the idea behind these checks. I'll refer to the new board state as New and the canonical board it is being compared to as
Canonical.
1. Check if the center spot of New is equal to the center spot of Canonical, if yes move on
2. Check if the top left corner (spot 1) of New is equal to any of the corners of Canonical (spots 1, 3, 7, 9), if yes move on
3. Check if the bottom right corner (spot 9) of New is equal to the opposite of the matching corner from the previous step of Canonical (9 if the
    previous step was 1, 7 if the previous step was 3, 3 if previous step was 7, 1 if previous step was 9), if yes move on
4. Create 2 separate checks that checks if the top right corner (spot 3) of New is equal to either of the remaining corners of Canonical that are not
    matched to 1 and 9 of New, if yes for either, move on to their respective if statement
5. Check if the entire board of New matches the entire board of Canonical using the correct transformation of New based on where spots 1, 3, 9 of
    New match to Canonical, if yes then New is not canonical
'''
def isNew(board):
    for canonical in canonical_list:
        # np.where(board[0:12] == 5)[0] will return an array of all the indeces in board[0:12] that are equal to 5
        # This allows us to prove equality even if there are multiple pieces on that spot
        if np.where(board[0:12] == 5)[0] != np.where(canonical[0:12] == 5)[0]:
             # The key word 'continue' means to exit the current iteration of the for-loop and continue to the next iteration
             continue
        
        elif np.where(board[0:12] == 1)[0] == np.where(canonical[0:12] == 1)[0]:
            if np.where(board[0:12] == 9)[0] == np.where(canonical[0:12] == 9)[0]:
                if np.where(board[0:12] == 3)[0] == np.where(canonical[0:12] == 3)[0]:
                    if board == canonical:
                        return False
                    else:
                        continue
                elif np.where(board[0:12] == 3)[0] == np.where(canonical[0:12] == 7)[0]:
                    if np.append(r90_hreflect_mapping[board[:12]], board[12:]) == canonical:
                        return False
                    else:
                        continue
                else:
                    continue
            else:
                continue
        
        elif np.where(board[0:12] == 1)[0] == np.where(canonical[0:12] == 3)[0]:
            if np.where(board[0:12] == 9)[0] == np.where(canonical[0:12] == 7)[0]:
                if np.where(board[0:12] == 3)[0] == np.where(canonical[0:12] == 9)[0]:
                    if np.append(rotation_90_mapping[board[12:]], board[12:]) == canonical:
                        return False
                    else:
                        continue
                elif np.where(board[0:12] == 3)[0] == np.where(canonical[0:12] == 1)[0]:
                    if np.append(horizontal_reflection_mapping[board[:12]], board[12:]) == canonical:
                        return False
                    else:
                        continue
                else:
                    continue
            else:
                continue
            
        elif np.where(board[0:12] == 1)[0] == np.where(canonical[0:12] == 7)[0]:
            if np.where(board[0:12] == 9)[0] == np.where(canonical[0:12] == 3)[0]:
                if np.where(board[0:12] == 3)[0] == np.where(canonical[0:12] == 1)[0]:
                    if rotation_270_mapping[board] == canonical:
                        return False
                    else:
                        continue
                elif np.where(board[0:12] == 3)[0] == np.where(canonical[0:12] == 9)[0]:
                    if vertical_reflection_mapping[board] == canonical:
                        return False
                    else:
                        continue
                else:
                    continue
            else:
                continue
            
        elif np.where(board[0:12] == 1)[0] == np.where(canonical[0:12] == 9)[0]:
            if np.where(board[0:12] == 9)[0] == np.where(canonical[0:12] == 1)[0]:
                if np.where(board[0:12] == 3)[0] == np.where(canonical[0:12] == 7)[0]:
                    if rotation_180_mapping[board] == canonical:
                        return False
                    else:
                        continue
                elif np.where(board[0:12] == 3)[0] == np.where(canonical[0:12] == 3)[0]:
                    if r90_vreflect_mapping[board] == canonical:
                        return False
                    else:
                        continue
                else:
                    continue
            else:
                continue
        else:
            continue
        
    return True
        

In [54]:
test_board = np.array([1, 9, 3, 5, 0, 0, 8, 2, 0, 4, 1, 9, 0, 0])
test_board2 = np.array([0, 7, 5, 3, 0, 9, 0, 0, 4, 0, 0, 0, 0, 0])

In [55]:
test_board = test_board.reshape(7,2)
test_board2 = test_board2.reshape(7, 2)

In [56]:
np.where(test_board[0:6] == 5)[0]

array([1], dtype=int64)

In [57]:
if np.all(np.where(test_board[0:6] == 1)[0] == np.where(test_board2[0:6] == 1)[0]):
    print('True')
else:
    print('False')

False


In [58]:
if np.array_equal(np.where(test_board[0:6] == 1)[0] == np.where(test_board2[0:6] == 1)[0]):
    print('True')
else:
    print('False')

TypeError: array_equal() missing 1 required positional argument: 'a2'

In [59]:
test_board[0:6]

array([[1, 9],
       [3, 5],
       [0, 0],
       [8, 2],
       [0, 4],
       [1, 9]])

In [60]:
winning_patterns = [
    np.array([1, 2, 3]),
    np.array([4, 5, 6]),
    np.array([7, 8, 9]),
    np.array([1, 4, 7]),
    np.array([2, 5, 8]),
    np.array([3, 6, 9]),
    np.array([1, 5, 9]),
    np.array([3, 5, 7])
]

In [61]:
def check_win(board):
    winner0 = None
    winner1 = None
    # Checking win conditions for player0
    for win_index in np.where(np.all(np.isin(winning_patterns, board[0:6]), axis = 1))[0]:
        winner0 = True
        for p1_index in (np.where(np.isin(board[6:12], winning_patterns[win_index]))[0]):
            if p1_index > np.where(np.isin(board[0:6], board[6:12][p1_index]))[0][-1]:
                winner0 = False
                break
        if winner0:
            board[13] = 10
            break

    for win_index in np.where(np.all(np.isin(winning_patterns, board[6:12]), axis = 1))[0]:
        winner1 = True
        for p0_index in (np.where(np.isin(board[0:6], winning_patterns[win_index]))[0]):
            if p0_index > np.where(np.isin(board[6:12], board[0:6][p0_index]))[0][-1]:
                winner1 = False
                break
        if winner1:
            board[13] = -10 if winner0 != True else 500
    
    return board

In [62]:
'''
The legal_moves_set() function takes in a board and determines all the possible moves that the current player could make. It first creates a copy of
the current board that is taken in and assigns it to temp_board so we can make changes to the temp_board without affecting the original board. It calls
the available_pieces() and eliminate_redundant_pieces() functions to see what pieces are available to be played, then for each available piece it loops
through spots 1-9 on the board and determines how many of the 9 spots that piece could be played (depending on what pieces already occupy those spots).
If the function finds a legal move then it makes that move on the temp_board, then submits that temp_board with the new move to the check_symmetry()
function so that we can check if this board or a board symmetric to it already exists in canonical_boards. The check_symmetry() function will then add
the board to canonical_boards if it is a canonical board. Then the temp_board variable is re-assigned to the original board and the loop continues to
the next spots and pieces to check for more legal moves. 
'''
def legal_moves_set(board):
    temp_board = board.copy()
    pieces  = eliminate_redundant_pieces(available_pieces(board))
    # Loop through all available pieces that can be played
    for piece in pieces:
        # Loop through numbers 1-9 which represent the 9 spots on the board
        for spot in range(1, 10):
            # Checks to see if the current piece is a small piece, meaning that a spot must be empty for it to be played
            if piece in (0, 1, 6, 7):
                # Checks to see if any piece is on the current spot of the loop
                if spot in board[0:12]:
                    continue
                else:
                    # Assigns the spot to the index of the piece in temp_board because it is confirmed to be a legal move
                    # This action can be looked at as the current player making a move
                    temp_board[piece] = spot
                    # Since each player has two pieces of the same size, 2 boards that are exactly the same could both be added to canonical_boards if
                    # the two boards are the exact same except the two same-sized pieces are swapped. The pieces have different index representations
                    # in our model but represent the same piece. This if-statement checks to see if the index of the current piece in the for-loop is
                    # even or odd, then sorts it and its matching piece in increasing order. If board[0] = 9 and board[1] = 3 then board[0] is now 3
                    # and board[1] is now 9.
                    if piece % 2 == 1:
                        temp_board[piece-1:piece+1].sort()
                    elif piece % 2 == 0 and temp_board[piece+1] != 0:
                        temp_board[piece:piece+2].sort()
                    temp_board = check_win(temp_board)
                    check_symmetry(temp_board)
                    temp_board = board.copy()
            # Checks to see if the current piece is a medium piece, so it can be played in an empty spot or on top of a small piece
            elif piece in (2, 3, 8, 9):
                # np.where checks to see if there are any pieces in the original board that currently occupy the spot we are looking at. np.isin then
                # checks if any of those pieces in the spot are medium or large. Finally, np.any returns True if there are pieces in the spot that are
                # medium/large and False if there are no pieces in the spot or the piece in the spot is small.
                if np.any(np.isin(np.where(board[0:12] == spot)[0], (2, 3, 8, 9, 4, 5, 10, 11))):
                    continue
                else:
                    temp_board[piece] = spot
                    if piece % 2 == 1:
                        temp_board[piece-1:piece+1].sort()
                    elif piece % 2 == 0 and temp_board[piece+1] != 0:
                        temp_board[piece:piece+2].sort()
                    temp_board = check_win(temp_board)
                    check_symmetry(temp_board)
                    temp_board = board.copy()
            else:
                # Returns True if there are any pieces in the spot we are looking at and the piece is large, returns False if there are no pieces in the
                # spot or if there are pieces in the spot but they are small or medium.
                if np.any(np.isin(np.where(board[0:12] == spot)[0], (4, 5, 10, 11))):
                    continue
                else:
                    temp_board[piece] = spot
                    if piece % 2 == 1:
                        temp_board[piece-1:piece+1].sort()
                    elif piece % 2 == 0 and temp_board[piece+1] != 0:
                        temp_board[piece:piece+2].sort()
                    temp_board = check_win(temp_board)
                    check_symmetry(temp_board)
                    temp_board = board.copy()

In [63]:
'''
The legal_moves_list() function is an exact copy of legal_moves_set() except it will be used for our 'list' approach, so the only difference is that
isNew() is used instead of check_symmetry() and new boards are added to canonical_list instead of canonical_boards.
'''
def legal_moves_list(board):
    temp_board = board.copy()
    pieces  = eliminate_redundant_pieces(available_pieces(board))
    for piece in pieces:
        for spot in range(1, 10):
            if piece in (0, 1, 6, 7):
                if spot in board[0:12]:
                    continue
                else:
                    temp_board[piece] = spot
                    if piece % 2 == 1:
                        temp_board[piece-1:piece+1].sort()
                    elif piece % 2 == 0 and temp_board[piece+1] != 0:
                        temp_board[piece:piece+2].sort()
                    temp_board = check_win(temp_board)
                    temp_board = temp_board.reshape(7, 2)
                    # isNew() function returns True if the board is canonical and False if the board or a symmetric version of it is found in
                    # canonical_list. Therefore we must append the board here if isNew() is True.
                    if isNew(temp_board): canonical_list.append(temp_board)
                    temp_board = board.copy()
            elif piece in (2, 3, 8, 9):
                if np.any(np.isin(np.where(board[0:12] == spot)[0], (2, 3, 8, 9, 4, 5, 10, 11))):
                    continue
                else:
                    temp_board[piece] = spot
                    if piece % 2 == 1:
                        temp_board[piece-1:piece+1].sort()
                    elif piece % 2 == 0 and temp_board[piece+1] != 0:
                        temp_board[piece:piece+2].sort()
                    temp_board = check_win(temp_board)
                    temp_board = temp_board.reshape(7, 2)
                    if isNew(temp_board): canonical_list.append(temp_board)
                    temp_board = board.copy()
            else:
                if np.any(np.isin(np.where(board[0:12] == spot)[0], (4, 5, 10, 11))):
                    continue
                else:
                    temp_board[piece] = spot
                    if piece % 2 == 1:
                        temp_board[piece-1:piece+1].sort()
                    elif piece % 2 == 0 and temp_board[piece+1] != 0:
                        temp_board[piece:piece+2].sort()
                    temp_board = check_win(temp_board)
                    temp_board = temp_board.reshape(7, 2)
                    if isNew(temp_board): canonical_list.append(temp_board)
                    temp_board = board.copy()

In [ ]:
canonical_boards.add(tuple(np.array([1, 9, 0, 0, 0, 0, 2, 0, 0, 0, 9, 0, 0, 0])))
canonical_boards

{(1, 9, 0, 0, 0, 0, 2, 0, 0, 0, 9, 0, 0, 0)}

In [ ]:
exception_board = np.array([1, 9, 0, 0, 0, 0, 8, 0, 0, 0, 1, 0, 0, 0])

In [ ]:
check_symmetry(exception_board)

In [ ]:
canonical_list.append(np.array([1, 9, 0, 0, 0, 0, 2, 0, 0, 0, 9, 0, 0, 0]))

In [ ]:
isNew(exception_board)

C:\Users\mjw01\AppData\Local\Temp\ipykernel_4892\988235250.py:28: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.where(board[0:12] == 5)[0] != np.where(canonical[0:12] == 5)[0]:


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
middle_game_state = np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])
placeable_pieces = available_pieces(middle_game_state)
leftover_pieces = eliminate_redundant_pieces(available_pieces(middle_game_state))

In [ ]:
print(f'placeable_pieces = {placeable_pieces}\n'
      f'leftover_pieces = {leftover_pieces}')

placeable_pieces = [0 1 5]
leftover_pieces = [0 5]


In [ ]:
copy = np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])
symmetric1 = np.append(rotation_90_mapping[np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])[:12]], np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])[12:])
symmetric2 = np.append(rotation_180_mapping[np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])[:12]], np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])[12:])
symmetric3 = np.append(rotation_270_mapping[np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])[:12]], np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])[12:])
symmetric4 = np.append(vertical_reflection_mapping[np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])[:12]], np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])[12:])
symmetric5 = np.append(horizontal_reflection_mapping[np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])[:12]], np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])[12:])
symmetric6 = np.append(r90_vreflect_mapping[np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])[:12]], np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])[12:])
symmetric7 = np.append(r90_hreflect_mapping[np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])[:12]], np.array([0, 0, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0])[12:])

In [ ]:
legal_moves_set(middle_game_state)

In [ ]:
canonical_boards

{(0, 0, 3, 8, 1, 5, 0, 0, 9, 2, 3, 0, 0, 0),
 (0, 0, 3, 8, 2, 5, 0, 0, 9, 2, 3, 0, 0, 0),
 (0, 0, 3, 8, 2, 5, 0, 0, 9, 2, 3, 0, 0, 10),
 (0, 0, 3, 8, 4, 5, 0, 0, 9, 2, 3, 0, 0, 0),
 (0, 0, 3, 8, 5, 6, 0, 0, 9, 2, 3, 0, 0, 0),
 (0, 0, 3, 8, 5, 7, 0, 0, 9, 2, 3, 0, 0, 0),
 (0, 0, 3, 8, 5, 8, 0, 0, 9, 2, 3, 0, 0, 0),
 (0, 0, 3, 8, 5, 9, 0, 0, 9, 2, 3, 0, 0, 0),
 (0, 1, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0),
 (0, 4, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0),
 (0, 6, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0),
 (0, 7, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0)}

In [ ]:
legal_moves_list(middle_game_state)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
canonical_list

[array([0, 1, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0]),
 array([0, 4, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0]),
 array([0, 6, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0]),
 array([0, 7, 3, 8, 5, 0, 0, 0, 9, 2, 3, 0, 0, 0]),
 array([0, 0, 3, 8, 1, 5, 0, 0, 9, 2, 3, 0, 0, 0]),
 array([ 0,  0,  3,  8,  2,  5,  0,  0,  9,  2,  3,  0,  0, 10]),
 array([0, 0, 3, 8, 4, 5, 0, 0, 9, 2, 3, 0, 0, 0]),
 array([0, 0, 3, 8, 5, 6, 0, 0, 9, 2, 3, 0, 0, 0]),
 array([0, 0, 3, 8, 5, 7, 0, 0, 9, 2, 3, 0, 0, 0]),
 array([0, 0, 3, 8, 5, 8, 0, 0, 9, 2, 3, 0, 0, 0]),
 array([0, 0, 3, 8, 5, 9, 0, 0, 9, 2, 3, 0, 0, 0])]

In [ ]:
board1 = create_board()
board1[1] = 5
board1[0] = 9
board1[2] = 0
board1[3] = 3
board1[4] = 7
board1[5] = 1
board1[11] = 5
board1[6] = 4
board1[8] = 6
board1

array([9, 5, 0, 3, 7, 1, 4, 0, 6, 0, 0, 5, 0, 0])

In [ ]:
check_win(board1)
board1

array([  9,   5,   0,   3,   7,   1,   4,   0,   6,   0,   0,   5,   0,
       -10])

In [64]:
board1 = np.array([0, 0, 8, 0, 1, 2, 6, 0, 7, 0, 3, 0, 0, 0])
board2 = np.array([3, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0])
board3 = np.array([0, 0, 0, 0, 5, 8, 0, 0, 3, 0, 2, 0, 0, 0])

In [66]:
legal_moves_list(board2)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()